In [1]:
import pandas as pd

In [2]:
chunk_size = 10000  # Adjust the chunk size as needed
all_chunks = []

for i in range(1,8):
    chunks = pd.read_csv(f'data/tweets_batch_00{i}.tsv', sep='\t',
                         usecols=["user_id", "tweet_id",
                                  "reply_to_user_id", "reply_to_tweet_id", "full_text"],
                         engine='python',
                         on_bad_lines='skip',
                         chunksize=chunk_size)
    all_chunks += chunks

df_replies = pd.concat(all_chunks, ignore_index=True)

In [3]:
def get_replies_tuit(tweet_id, user_id):
    # get replies for tuits in topic
    df_replies_tweet = df_replies[(df_replies.reply_to_tweet_id == tweet_id) & (df_replies.user_id != user_id)]

    return df_replies_tweet

In [5]:
def get_engaging_tuits(n_replies=10):
    # with at least one reply
    have_replies = df_replies.reply_to_tweet_id.unique()
    df_tuits_replied = df_replies[df_replies.tweet_id.isin(have_replies)]

    df_tuits_replied["reply_count"] = df_tuits_replied.apply(lambda x: len(get_replies_tuit(x.tweet_id, x.user_id)), axis=1)

    df_tuits_engaging = df_tuits_replied[df_tuits_replied.reply_count >= n_replies]
    return df_tuits_engaging

In [14]:
df_tuits_engaging = get_engaging_tuits(n_replies=3)

/tmp/ipykernel_3850211/3138283614.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tuits_replied["reply_count"] = df_tuits_replied.apply(lambda x: len(get_replies_tuit(x.tweet_id, x.user_id)), axis=1)


In [15]:
df_tuits_engaging.shape

(239, 6)

In [16]:
df_tuits_engaging.reply_count.sum()

np.int64(1529)

In [17]:
df_tuits_engaging.to_csv("data/engaging_tuits.csv", index=False, sep="\t")

In [18]:
pd.read_csv("data/engaging_tuits.csv", sep="\t").shape

(239, 6)

In [21]:
df_replies_to_engaging = df_replies[
    (df_replies.reply_to_tweet_id.isin(df_tuits_engaging.tweet_id.unique()))
    & (df_replies.user_id != df_replies.reply_to_user_id)
]

In [22]:
df_replies_to_engaging.shape

(1529, 5)

In [23]:
df_replies_to_engaging.to_csv("data/replies_to_engaging_tuits.csv", index=False, sep="\t")

In [24]:
pd.read_csv("data/replies_to_engaging_tuits.csv", sep="\t").shape

(1529, 5)